In [1]:
import numpy as np
from keras.applications import vgg16
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from IPython.display import Image
import cv2 as cv
import time
from scipy.optimize import fmin_l_bfgs_b

Using TensorFlow backend.


In [2]:
# Import Base and Style images and resize them into same dimension
base_temp = cv.imread("base.jpg")
base_img = cv.resize(base_temp, dsize=(800, 600), interpolation=cv.INTER_CUBIC)

style_temp = cv.imread("style.jpg")
style_img = cv.resize(style_temp, dsize=(800, 600), interpolation=cv.INTER_CUBIC)

In [3]:
# Add one more dimension to concatinate the images
base_image = K.variable(np.array([base_img]))
style_image = K.variable(np.array([style_img]))

print(np.shape(base_image))
print(np.shape(style_image))

(1, 600, 800, 3)
(1, 600, 800, 3)


In [4]:
# Create combined image as placeholder 
combined_image = K.placeholder((1,600,800,3))
# Combine all data to run it with VGG16
input_tensor = K.concatenate([base_image, style_image, combined_image], axis=0)

In [5]:
# Create instance of pre-trained VGG16 Classifier model, do not include last layer
model = vgg16.VGG16(input_tensor=input_tensor, weights="imagenet", include_top=False)
print("Model Loaded")

Model Loaded


In [6]:
# Layers in VGG16 listed
layers = dict([(layer.name, layer.output) for layer in model.layers])

In [7]:
# Weights of losses, these are hyperparameters of our algorithm
content_weight = 0.025
style_weight = 5.0
total_variation_weight = 1.0

In [8]:
########################   C O N T E N T   L O S S   ###########################

In [9]:
# Calculate the Content Loss, extract the activations from CNN
layer_features = layers['block2_conv2']
content_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]

# Calculate the OLS error between combined image and content image
content_loss = K.sum(K.square(combination_features - content_image_features))
loss = K.variable(0.)
loss = content_weight * content_loss

In [10]:
#########################   S T Y L E   L O S S   #############################

In [11]:
# Calculate Gram Matrix of given matrix (X*X^T)
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [12]:
def style_loss(style, combination):
    style_gram = gram_matrix(style)
    content_gram = gram_matrix(combination)
    channels = 3
    size = 600 * 800
    return K.sum(K.square(style_gram - content_gram)) / (4. * (channels ** 2) * (size ** 2))

# Layers used to calculate style loss
feature_layers = ['block1_conv2', 'block2_conv2',
                  'block3_conv3', 'block4_conv3',
                  'block5_conv3']

# Sum the losses we get from each layer for the style loss
for layer_name in feature_layers:
    layer_features = layers[layer_name]
    style_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl

In [13]:
def total_variation_loss(x):
    a = K.square(x[:, :600-1, :800-1, :] - x[:, 1:, :800-1, :])
    b = K.square(x[:, :600-1, :800-1, :] - x[:, :600-1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

loss += total_variation_weight * total_variation_loss(combined_image)

In [14]:
# Calculate gradients of the total loss w.r.t. combined image
gradients = K.gradients(loss, combined_image)

In [15]:
############################ I GIVE UP HERE ####################################

In [17]:
outputs = [loss]
outputs += gradients
f_outputs = K.function([combined_image], outputs)

def eval_loss_and_grads(x):
    x = x.reshape((1, 600, 800, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values

class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

evaluator = Evaluator()

In [ ]:
x = np.random.uniform(0, 255, (1, 600, 800, 3)) - 128.

iterations = 10

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
